In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['sample_submission.csv', 'test.csv', 'train.csv']


In [2]:
### Here I will work with two convulation layer and two full connected layer. The idea is just to get familiar with tensorflow
df=pd.read_csv("../input/train.csv")
dftest=pd.read_csv("../input/test.csv")

In [3]:
dftest.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
import tensorflow as tf

In [5]:
ytrain = df['label']

ytrain = tf.one_hot(ytrain, depth=10)
y_oh_train = tf.Session().run(ytrain).astype(dtype=np.float64)




In [6]:
## reshaping to a tensor
x_train=df.drop(labels=["label"],axis=1)

x_train=x_train.values.astype(dtype=np.float32)
x_train=x_train.reshape(42000,28,28,1)




In [7]:
### Creating graph
X= tf.placeholder(tf.float32,[None,x_train.shape[1],x_train.shape[2],x_train.shape[3]])
Y_=tf.placeholder(tf.float32, [None,10] )
lr=tf.placeholder(tf.float32)

In [8]:
#We will define that first convulation has 5 filters and will be 3x3 and the second one has 9 filters and will be 3x3
C1=6
C2=7
C3=8
FC=200


In [9]:
# Random initialization of the parameters first convulation
W1=tf.Variable(tf.truncated_normal([5, 5, 1, C1], stddev=0.1))
B1=tf.Variable(tf.constant(0,tf.float32,[C1]))
               
# Random initialization of the parameters second convulation
W2=tf.Variable(tf.truncated_normal([4, 4, C1, C2], stddev=0.1))
B2=tf.Variable(tf.constant(0,tf.float32,[C2]))          
               
# Random initialization of the parameters third conv
W3=tf.Variable(tf.truncated_normal([4,4,C2,C3],stddev=0.1))
B3=tf.Variable(tf.constant(0,tf.float32,[C3]))        
              
W4=tf.Variable(tf.truncated_normal([7*7*C3,FC],stddev=0.01))
B4=tf.Variable(tf.constant(0,tf.float32,[FC]))             

W5=tf.Variable(tf.truncated_normal([FC,10 ],stddev=0.01))
B5=tf.Variable(tf.constant(0,tf.float32,[10])) 
               

In [10]:
##Model 
#First convolution
stride=1
Y1=tf.nn.relu(tf.nn.conv2d(X,W1,strides=[1,stride,stride,1],padding="SAME")+B1)
 
#second convolution
stride=2
Y2=tf.nn.relu(tf.nn.conv2d(Y1,W2,strides=[1,stride,stride,1],padding="SAME")+B2)

#third convolution
stride=2
Y3=tf.nn.relu(tf.nn.conv2d(Y2,W3,strides=[1,stride,stride,1],padding="SAME")+B3)

#Reshape Y2 for fully connected layer
Yfc=tf.reshape(Y3,[-1,7*7*C3])

Y4=tf.nn.relu(tf.matmul(Yfc,W4)+B4)
Y4=tf.matmul(Y4,W5)+B5
Y=tf.nn.softmax(Y4)

In [11]:
#Cost function
cross_entropy=tf.nn.softmax_cross_entropy_with_logits_v2(logits=Y4,labels=Y_)
cross_entropy=tf.reduce_mean(cross_entropy)*100

# accuracy of the trained model, between 0 (worst) and 1 (best)
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
predictions = tf.argmax(Y, 1)



train_nn=tf.train.AdamOptimizer(lr).minimize(cross_entropy)

In [12]:
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)

def getBatch(i, size, x_train, y_oh_train):
    startIndex = (i * size) % x_train.shape[0]
    endIndex = startIndex + size
    batch_X = x_train[startIndex : endIndex]
    batch_Y = y_oh_train[startIndex : endIndex]
    return batch_X, batch_Y

def training_step(i,learning_rate,size):
    batch_X, batch_Y = getBatch(i, size, x_train, y_oh_train)
    
    sess.run(train_nn,{X:batch_X,Y_:batch_Y,lr:learning_rate})




In [13]:
for i in range(10000+1):
    training_step(i,0.003,100)

In [14]:
# Print the training accuracy
a, c = sess.run([accuracy, cross_entropy], {X: x_train , Y_: y_oh_train})
print("\n ********* training accuracy:" + str(a) + " training loss: " + str(c) )



 ********* training accuracy:0.98940474 training loss: 5.514434
